# DSPy Self-Consistency

In [1]:
#|default_exp dspy.module.sc

In [2]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [3]:
#|export
from copy import deepcopy

import dspy
from dspy.primitives.program import Module

from bellem.logging import get_logger

log = get_logger(__name__)

In [4]:
#|export

class SelfConsistency(Module):
    def __init__(self, predictors: list, field: str):
        super().__init__()
        assert len(predictors) > 1, "Self-consistency prompting requires at least 2 predictors"
        self.predictors = predictors
        self.field = field
    
    @classmethod
    def from_clones(cls, predictor, field: str, n: int):
        return cls(predictors=[deepcopy(predictor) for _ in range(n)], field=field)    

    def forward(self, **kwargs):
        predictions = [predictor(**kwargs) for predictor in self.predictors]
        final_prediction = dspy.majority(predictions, field=self.field)
        final_prediction.candidate_predictions = predictions
        return final_prediction

In [5]:
#|hide
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from bellem.dspy.utils import configure_lm

lm = configure_lm('openai/gpt-4o-mini', temperature=0.7)

In [7]:
context = """
# Corn production in the United States\nIowa, the largest producer of corn in the US, grows three times as much corn as Mexico. Iowa harvested 3,548 acres (1,436 ha) of sweet corn in 2007. In 2011, the state had 92,300 corn farms on 30,700,000 acres (12,400,000 ha), the average size being 333 acres (135 ha), and the average dollar value per acre being US $6,708. In the same year, there were 13.7 million harvested acres of corn for grain, producing 2.36 billion bushels, which yielded 172.0 bu / acre, with US $14.5 billion of corn value of production. Almost 1.88 billion bushels of corn were grown in the state in 2012 on 13.7 million acres of land, while the 2013 projections are 2.45 billion bushels of corn on 13.97 million acres of land.\n\n# Iowa\nAlmost immediately after achieving territorial status, a clamor arose for statehood. On December 28, 1846, Iowa became the 29th state in the Union when President James K. Polk signed Iowa's admission bill into law. Once admitted to the Union, the state's boundary issues resolved, and most of its land purchased from the Indians, Iowa set its direction to development and organized campaigns for settlers and investors, boasting the young frontier state's rich farmlands, fine citizens, free and open society, and good government.
""".strip()

question = 'Who was president when the place where the majority of sweet corn is grown became a state?'

In [8]:
class QA(dspy.Signature):
    """Answer the question based on the given context."""

    context: str = dspy.InputField()
    question = dspy.InputField()
    answer = dspy.OutputField(desc="in a few words")

cot = dspy.ChainOfThought(QA)
sc = SelfConsistency.from_clones(cot, field='answer', n=3)

In [9]:
sc_prediction = sc(context=context, question=question)
sc_prediction

Prediction(
    reasoning='Iowa, where the majority of sweet corn is grown, became a state on December 28, 1846. At that time, James K. Polk was the President of the United States, having taken office in 1845. Therefore, he was the president when Iowa achieved statehood.',
    answer='James K. Polk',
    candidate_predictions=[Prediction(
    reasoning='Iowa, where the majority of sweet corn is grown, became a state on December 28, 1846. At that time, James K. Polk was the President of the United States, having taken office in 1845. Therefore, he was the president when Iowa achieved statehood.',
    answer='James K. Polk'
), Prediction(
    reasoning="Iowa became a state on December 28, 1846. At that time, the President of the United States was James K. Polk, who signed Iowa's admission bill into law. Therefore, James K. Polk was the president when Iowa, the largest producer of sweet corn in the U.S., became a state.",
    answer='James K. Polk'
), Prediction(
    reasoning="Iowa, the 

In [10]:
#|hide
import nbdev; nbdev.nbdev_export()